# FlashAttention 加速原理简介

# 前言
FlashAttention是对如下计算公式的优化(忽略$\frac{1}{\sqrt D})$  
$ O = softmax(QK^{T})V $

In [1]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/2205.14135', width=1000, height=700)

# Online softmax
## Raw softmax
原始的softmax定义如下  
$softmax(\{x_{1},...x_{N}\}) = \{\frac{e^{x_{i}}}{\sum_{j=1}^{N} e^{x_{j}}}\}^{N}_{i=1}$  
由于fp16的最大值为66536，那么也就是说当指数大于11时，$e^{12} \approx 162754$就会触发fp16的上溢出。

## Safe softmax
为了解决如上指数带来的易溢出问题，safe softmax做出如下改动  
$softmax(\{x_{1},...x_{N}\}) = \{\frac{e^{x_{i}  -  m}}{\sum_{j=1}^{N} e^{x_{j}  -  m}}\}^{N}_{i=1}$   
$m = max_{j=1}^{N}(x_{j})$  
由于$ x_{i} - m \leq 0 $,上溢出的问题也就解决了。

## Online softmax
似乎softmax的问题都被Safe softmax解决了，那么Online softmax解决的是什么问题呢？  

假设我们有一个长度大到内存装不下的向量，如何计算这个向量的softmax向量呢？  

In [40]:
import torch
x = torch.randn([100])
max_item = torch.max(x, -1)
print(max_item[0])
sum_value = torch.sum(torch.exp(x - max_item[0]))
print(sum_value)
result = torch.exp(x - max_item[0])/sum_value
print(result)

result2 = torch.softmax(x, -1)
print(torch.allclose(result, result2))

tensor(2.3544)
tensor(14.0354)
tensor([0.0052, 0.0265, 0.0104, 0.0075, 0.0201, 0.0187, 0.0140, 0.0142, 0.0327,
        0.0014, 0.0069, 0.0106, 0.0171, 0.0034, 0.0126, 0.0370, 0.0129, 0.0033,
        0.0073, 0.0047, 0.0008, 0.0015, 0.0046, 0.0712, 0.0029, 0.0163, 0.0012,
        0.0072, 0.0061, 0.0255, 0.0029, 0.0087, 0.0030, 0.0084, 0.0062, 0.0016,
        0.0354, 0.0174, 0.0033, 0.0082, 0.0028, 0.0016, 0.0055, 0.0114, 0.0013,
        0.0009, 0.0074, 0.0067, 0.0045, 0.0133, 0.0034, 0.0209, 0.0026, 0.0154,
        0.0014, 0.0052, 0.0090, 0.0146, 0.0034, 0.0007, 0.0079, 0.0219, 0.0434,
        0.0095, 0.0131, 0.0024, 0.0109, 0.0050, 0.0009, 0.0038, 0.0247, 0.0093,
        0.0036, 0.0125, 0.0030, 0.0114, 0.0043, 0.0051, 0.0109, 0.0012, 0.0043,
        0.0028, 0.0025, 0.0165, 0.0034, 0.0169, 0.0021, 0.0020, 0.0044, 0.0099,
        0.0034, 0.0122, 0.0122, 0.0023, 0.0484, 0.0128, 0.0020, 0.0031, 0.0009,
        0.0023])
True


首先，直觉就是分治，切片。  
Step1:切片，求出各切片的最大值后得出整个向量最大值。  
Step2:切片，求出个切片的sum_value值后得到整个向量的sum_value。  
Step3:切片，分治得到整个向量的softmax result。  

无论是上文中的代码的计算逻辑还是切片，都需要遍历完整向量3次，I/O效率很低。

切片的Step2计算公式为(假设为第一块切片，j从1开始)  
$ sum\_value^{'} = \sum_{j=1}^{i} e^{x_{j}  -  m_{N}}$  
$m_{N} = max_{j=1}^{N}(x_{j})$ 

就是由$m_{N}$对$N$的依赖，导致无法将Step1-3融合在一起。  
如果将上文公式变换成如下形式，那么我们就解除了对$N$的依赖，副作用是，max值并非全局最大。  
$ sum\_value^{'} = \sum_{j=1}^{i} e^{x_{j}  -  m_{i}}$  
$m_{i} = max_{j=1}^{i}(x_{j})$ 

$ sum\_value^{''} = \sum_{j=1}^{i+1} e^{x_{j}  -  m_{i+1}} \\ \quad\quad\qquad\  = (\sum_{j=1}^{i} e^{x_{j}  -  m_{i+1}}) + e^{x_{i+1}  -  m_{i+1}}  \\ \quad\quad\qquad\ =  (\sum_{j=1}^{i} e^{x_{j}  -  m_{i}})e^{m_{i}-m_{i+1}} + e^{x_{i+1}  -  m_{i+1}} \\ \quad\quad\qquad\ =  sum\_value^{'}e^{m_{i}-m_{i+1}} + e^{x_{i+1}  -  m_{i+1}} $  

In [46]:
import numpy as np
global_max_item = -np.inf
prev_max_item = -np.inf
global_sum_value = 0
x1 = x[0 : 50]
x2 = x[50 : ]
for part in [x1, x2]:
    print(len(item))
    cur_max_item = torch.max(part, -1)[0]
    if cur_max_item > global_max_item:
        global_max_item = cur_max_item
    cur_sum_value = torch.sum(torch.exp(part - global_max_item))
    global_sum_value = global_sum_value * torch.exp(prev_max_item - global_max_item) + cur_sum_value
    prev_max_item = global_max_item
    
result3 = torch.exp(x - global_max_item) / global_sum_value
print(torch.allclose(result, result3))

50
50
True


# 参考资料
【1】FlashAttention 的速度优化原理是怎样的？ - Civ的回答 - 知乎
https://www.zhihu.com/question/611236756/answer/3132304304